<a href="https://colab.research.google.com/github/YvvYz/TTC/blob/main/TTC_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using public transportation in Toronto can be difficult due to the complicated routes and uncertain in arrival time of a bus. The actual arrival and departure time at a certain stop may vary depending on many factors, meaning that pure experience might not always work when planning the time to arrive at the bus stop. Machine learning algorithm is a usefull tool to analyze historical data of delay incidencies and find the key features that could be used to predict the delay time, especially when the amount of potential factors that could cause a delay incidence is massive.

The following sections illustrates complete process of data cleaning, feature engineering, model fitting and training.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_excel('ttc-bus-delay-data-2021.xlsx', sheet_name=None)
#File contains 12 spreadsheets, need to combine them.
df = pd.concat(data.values())

df.info()

The original data has 42269 records. Column description is as follows:

*   Date: Date on which the delay incidence is reported
*   Route: Route number
*   Time: Time when the delay incidence happened
*   Day: Day of week
*   Location: Location (name of bus stop) where the delay incidence happened
*   Incident: Cause of delay
*   Min Delay: Delay time in minute
*   Min Gap: Scheduled gap of time between current bus and the next bus in minute
*   Direction: Bound
*   Vehicle: Identifier of vehicle
*   Line: Same as Route (inconsistent column name)
*   Bound: Same as Direction (inconsistent column name)
*   Unnamed:10: Invalid

Combine columns that have inconsistent column names:

In [ ]:
df['Route'].fillna(df['Line'],inplace = True)
df['Direction'].fillna(df['Bound'],inplace = True)

# Cleaning/Feature Engineering - Min Delay & Delay Interval

In [ ]:
#delete rows that has no value in column "Min Delay"
df.dropna(subset=['Min Delay'])

Remove Outliers

In [ ]:
#box plot
sns.boxplot(x=df['Min Delay'])

Minutes Delay has some extremely large value, which might be errors generated when reporting the incidence, or indicating extreme situations such as service cancelled temporarily.

In [ ]:
#Calculate IQR
Q1 = df['Min Delay'].quantile(0.25)
Q3 = df['Min Delay'].quantile(0.75)
IQR = Q3 - Q1
print(IQR)

12.0


In [ ]:
#remove rows that contain outlier
df2 = df[ ~ ( (df['Min Delay'] < (Q1 - 1.5 * IQR)) | (df['Min Delay'] > (Q3 + 1.5 * IQR) ))]

In [ ]:
sns.boxplot(x=df2['Min Delay'])

Create bins for delay time, categorize delay time values as intervals.

(-1,0] contains only 0, meaning "On time", (0,5] means "Delay for 1-5 minutes".

In [ ]:
bin_delay = [-1,0,5,10,15,20,25,30,99999]

df2['Delay_interval']=pd.cut(df2['Min Delay'],bin_delay).astype('str')

In [ ]:
df2.info()

# Feature Engineering - Month

Create column "Month" to extract the month from date.

In [ ]:
df2['Month'] = df2['Date'].dt.month.astype('category')

In [ ]:
#Subset the dataframe so that only delay time greater than 0 is selected
delay = df2[df2['Min Delay'] > 0]

#count the number of rows for each month
month_count = delay['Month'].value_counts().sort_index()
#plot bar chart (month vs. frequency of delay)
plt.bar(x=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'],height=month_count)

> The frequencies of delay incidences have a seasonal pattern. Public transportation system can be influenced by factors such as weather and holiday seasons. Therefore month could be a feature to predict delay time.

# Cleaning/Feature Engineering - Route & Route Category

In [ ]:
#drop rows with no values in Route
df3 = df2.dropna(subset=['Route'])


Import a list of all regular routes and their service type. The file compiles routes information from TTC official site.

In [ ]:
route_all = pd.read_excel('TTC Routes.xlsx')

Create a new column "Route_Category" to record the service type of each record.

In [ ]:
#Subset the datafram based on value of route type, and add labels for routes.
regular = df3[df3['Route'].isin(route_all['Regular'])]
regular['Route_Category'] = 'Regular Route'

limited = df3[df3['Route'].isin(route_all['Limited'])]
limited['Route_Category'] = 'Limited Service'

night = df3[df3['Route'].isin(route_all['Night'])]
night['Route_Category'] = 'Night Service'

express = df3[df3['Route'].isin(route_all['Express'])]
express['Route_Category'] = 'Express Route'

community = df3[df3['Route'].isin(route_all['Community'])]
community['Route_Category'] = 'Community Bus'

streetcar = df3[df3['Route'].isin(route_all['StreetCar'])]
streetcar['Route_Category'] = 'Street Car'

streetcar_night = df3[df3['Route'].isin(route_all['StreetCar Night'])]
streetcar_night['Route_Category'] = 'Street Car'

#Combine subsets for all route types into one datafram
df4 = pd.concat([regular, limited, night, express, community, streetcar, streetcar_night])

In [ ]:
df4.info()

> Route is an important feature that can identify a bus trip. However, this feature will not provide predictive power if new data contains unknown, special or new routes.

> Route category identify the service type of this route. This column is created because it is possible that the delay times of different service are different. For example, trips during late night might encounter smaller vehicle volumn, express trips might have less stops and are les likely to delay.

# Cleaning/Feature Engineering - Time & Time Interval

The format of time in the original file is "hours : minute". Remove all special characters in time value and convert time to integer.

In [ ]:
df4 = df4.sort_values(['Time'])

In [ ]:
t = df4['Time'].astype('str')
t = t.str.replace(':','').astype(int)

Create bins for time, categorize time values as intervals.

(-1,100] means 0:00 to 1:00; (100,200] means 1:00 to 2:00, etc.

In [ ]:
bins = [-1,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2359]
df4['time_interval']=pd.cut(t,bins).astype('str')
df4['time_interval']

1794       (-1, 100]
1688       (-1, 100]
1710       (-1, 100]
1135       (-1, 100]
1300       (-1, 100]
            ...     
3388    (2300, 2359]
104     (2300, 2359]
3008    (2300, 2359]
2400    (2300, 2359]
1518    (2300, 2359]
Name: time_interval, Length: 39476, dtype: object

In [ ]:
delay = df4[df4['Min Delay']>0]
#plot histogram (time interval vs. delay frequency)
plt.hist(delay['time_interval'])

In [ ]:
df4.head()

> Traffic condition, which is very likely to influence bus schedule, usually follows certain pattern throughout a day. For example, traffic jam and collision usually happen more often during rush hours.

# Cleaning - Direction

The column "Direction" contains invalid and NA values.

In [ ]:
df5 = df4.dropna(subset=['Direction'])

Only keep rows with valid direction values.

In [ ]:
df5 = df5[df5['Direction'].isin(['N','E','W','S','B'])]

In [ ]:
df5['Direction'].unique().tolist()

# Day of Week

In [ ]:
#plot histogram (day of week vs. delay frequency)
delay = df5[df5['Min Delay']>0]
plt.hist(df5['Day'])

Delay frequency is higher during week days and lower during weekends (larger pedestrain and vehicle volume, higher needs for public transportation during weekdays). This indicates that delay time could also be influenced by day of week.

# Decision Tree/Random Forest 1

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

The first model contains features: month, time as intervals, day of the week, route type and direction.

In [ ]:
d = df5[['Month','time_interval','Day','Route_Category','Direction','Delay_interval']]
d.info()

In [ ]:
#One-hot-encode categorical feathers
data1 = pd.get_dummies(d[['Month','time_interval','Day','Route_Category','Direction']],drop_first=True)
features1 = data1.columns.values.tolist()

**Predict using decision tree**

In [ ]:
#define the model to use
tree1 = DecisionTreeClassifier(min_samples_split=100)

#subset data into training and testing data
X_train1, X_test1, y_train1, y_test1 = train_test_split(data1,d['Delay_interval'],test_size=0.15,random_state = 42)

In [ ]:
#train model and predict
tree1.fit(X_train1,y_train1)
y_pred1 = tree1.predict(X_test1)

#calculate accuracy
accuracy1  = accuracy_score(y_test1,y_pred1)
accuracy1

0.3884297520661157

**Predict using random forest**

Try different number limitation on depth, record each corresponding accuracy.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

#create empty list to store accuracy scores
accuracyList_1 = []
maxDepth_1 = []
for i in range(1,30):
  #define model to be used
  rf = RandomForestClassifier(criterion="entropy",max_depth=i, random_state=0)

  #train model
  rf.fit(X_train1,y_train1)
  #predict
  rf_pred = rf.predict(X_test1)
  #calculate accuracy
  rf_accuracy  = accuracy_score(y_test1,rf_pred)

  #record accuracy score
  accuracyList_1.append(rf_accuracy)
  maxDepth_1.append(i)


Plot accuracy vs. max depth

In [ ]:
plt.plot(maxDepth_1,accuracyList_1)
plt.xlabel('Maximum Depth')
plt.ylabel('Accuracy')
plt.show()

The accuracy of prediction reaches the highest value when the maximum depth is set to 16. Therefore use 16 as the max_depth.

In [ ]:
rf = RandomForestClassifier(criterion="entropy",max_depth=16, random_state=0)

rf.fit(X_train1,y_train1)
rf_pred = rf.predict(X_test1)
accuracy_score(y_test1,rf_pred)

0.41523886313243297

# KNN

This model uses KNN as algorith but with the same combination of features as decision tree 1.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#one-hot-encode categories
x = pd.get_dummies(d[['Month','time_interval','Day','Route_Category','Direction']],drop_first=True)
featuresK = x.columns.values.tolist()
#define target column
y = d['Delay_interval'].values

In [ ]:
#subset data into training and testing data
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.4,stratify=y,random_state=42)

Try different k value and find the one that yields the highest accuracy

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neighbors = np.arange(1,20)
train_accuracy = np.empty(len(neighbors))
test_accuracy = np.empty(len(neighbors))

for i,k in enumerate(neighbors):
    #define model to be used
    knn = KNeighborsClassifier(n_neighbors=k)
    #train model
    knn.fit(X_train,y_train)
    #calculate train accuracy
    train_accuracy[i] = knn.score(X_train,y_train)
    #calculate predict accuracy
    test_accuracy[i] = knn.score(X_test,y_test)

Plot training accuracy and testing accuracy vs. k

In [ ]:
plt.plot(neighbors,test_accuracy,label='Testing Accuracy')
plt.plot(neighbors,train_accuracy,label='Training Accuracy')
plt.legend()
plt.xlabel('Number of neighbors')
plt.ylabel('Accuracy')
plt.title('K-NN Varying number of neighbors')
plt.show()

As the plot shows, the accuracy of testing and training converge to about 40. It is reasonable to conclude that, with the current combination of features, KNN yields similar result to random forest or decision tree.



# Decision Tree/Random Forest 2

This model contains features: month, time intervals, day of week, route, route type and direction.

In [ ]:
d2 = df5[['Month','time_interval','Day','Route','Route_Category','Direction','Delay_interval']]

In [ ]:
data2 = pd.get_dummies(d2[['Month','time_interval','Day','Route','Route_Category','Direction']],drop_first=True)
features2 = data2.columns.values.tolist()

In [ ]:
data2.head()

Due to the high cardinality of data "route", the number of features is very large after one-hot-encoding.

**Predict using decision tree**

In [ ]:
tree2 = DecisionTreeClassifier(min_samples_split=100)

X_train2, X_test2, y_train2, y_test2 = train_test_split(data2,d2['Delay_interval'],test_size=0.15,random_state = 42)

In [ ]:
tree2.fit(X_train2,y_train2)
y_pred2 = tree2.predict(X_test2)

accuracy  = accuracy_score(y_test2,y_pred2)
accuracy

0.6182221326345495

**Predict using random forest**

In [ ]:
accuracyList_2 = []
maxDepth_2 = []
for i in range(80,150):
  rf = RandomForestClassifier(criterion="entropy",max_depth=i, random_state=0)

  rf.fit(X_train2,y_train2)
  rf_pred = rf.predict(X_test2)
  rf_accuracy  = accuracy_score(y_test2,rf_pred)

  accuracyList_2.append(rf_accuracy)
  maxDepth_2.append(i)

In [ ]:
plt.plot(maxDepth_2,accuracyList_2)
plt.xlabel('Maximum Depth')
plt.ylabel('Accuracy')
plt.show()

The highest accuracy score (64%) appears when maximum depth is at around 98. In this model the number of features are much bigger than random forest 1, this could lead to the result that the number of data records in each branch is much smaller in this model. The model might output biased result if the data size if not big enough.

# Decision Tree/Random Forest 3

This model contains features: month, time interval, day of week, route category, location and direction.

The model is built in comparison with the second decision tree/random forest model to compare the predictive power of "Route" and "Location". Since both features have large amount of classes, including both in a model will likely to make the model resource-consuming. Therefore, the purpose of this section is to discuss which of these two features should be chosen, if any.

In [ ]:
d3 = df5[['Month','time_interval','Day','Route_Category','Location','Direction','Delay_interval']]
d3.head()

,Month,time_interval,Day,Route_Category,Location,Direction,Delay_interval
1794,10,"(-1, 100]",Friday,Express Route,JANE AND TRETHEWEY,S,"(10, 15]"
1688,3,"(-1, 100]",Wednesday,Regular Route,LAWRENCE WEST STATION,E,"(15, 20]"
1135,11,"(-1, 100]",Sunday,Regular Route,WYNFORD AND DON MILLS,S,"(5, 10]"
1300,2,"(-1, 100]",Monday,Regular Route,MCNICOLL AND KENNEDY,E,"(25, 30]"
453,7,"(-1, 100]",Wednesday,Street Car,ST CLAIR AND OLD WESTO,E,"(10, 15]"


In [ ]:
data3 = pd.get_dummies(d3[['Month','time_interval','Day','Location','Route_Category','Direction']],drop_first=True)
features3 = data3.columns.values.tolist()

In [ ]:
data3.head()

,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,...,Direction_B,Direction_E,Direction_J,Direction_L,Direction_N,Direction_Q,Direction_R,Direction_S,Direction_T,Direction_W
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
tree3 = DecisionTreeClassifier(min_samples_split=100)

X_train3, X_test3, y_train3, y_test3 = train_test_split(data3,d3['Delay_interval'],test_size=0.15,random_state = 42)

In [ ]:
tree3.fit(X_train3,y_train3)

DecisionTreeClassifier(min_samples_split=100)

In [ ]:
y_pred3 = tree3.predict(X_test3)

In [ ]:
accuracy3 = accuracy_score(y_test3,y_pred3)
accuracy3

0.4708728079016327

In decision tree 2, the accuracy is 61% which is much higher than the accuracy score of this model, even that location provides more information of the specific location of a delay incidence. This might because that there are too many splits, and the model could overfit training data.

# Discussion and Conclusion

> Location data is not cleaned or chosen as feature for following reasons:

1.   TTC has over 5000 bus stops, one-hot-encoding this feature directly will generate too many variables. This can easily cause biased or extreme predicted value.
2.   Classifying location data needs additional information that has logical connection between bust stop names (geographic location, population density, traffic volume etc.), which is hard to find due to limited time and difficult to process.
3.   The data quality is relatively poor, some location class has incomplete stop name. Therefore it is very hard to map this column to other data.

With a good classification of location information, all models presented are likely to performe better. One potential way to classify bus stops is using classifier such as k-means or knn algorithm. Alternative method could be that using latitude and longtitude of each stop to put each stop into a geographic region.
> Among all tree-based models, the second one yields the highest accuracy. However, this model will lose predictive power if new route is added to the data. On the other hand, the first is not as accurate as the first one but it is more generally applicable.
